In [ ]:
from typing import Annotated, TypedDict, List

from llama_cpp import Llama

from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langchain_community.chat_models import ChatLlamaCpp
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables import Runnable

In [ ]:
chat: Runnable = ChatLlamaCpp(
    model_path="LlamaCppModels/Llama-3.2-1B-Instruct/Llama-3.2-1B-Instruct-Q6_K_L.gguf",
    n_ctx=4096,
    n_threads=8,
    temperature=0.1,
    model_kwargs={"chat_format": "llama-3"},
    verbose=False,
)


class ChatState(TypedDict):
    messages: Annotated[List[AnyMessage], "The running chat transcript"]


def call_model(state: ChatState) -> ChatState:
    reply = chat.invoke(state["messages"])
    return {"messages": state["messages"] + [reply]}


builder = StateGraph(ChatState)
builder.add_node("model", call_model)
builder.add_edge(START, "model")
builder.add_edge("model", END)

app = builder.compile()

In [ ]:
state: ChatState = {"messages": [HumanMessage(content="Where is Leipzig?.")]}

print("User:\n", state["messages"][-1].content)
state = app.invoke(state)
print("Assistant:\n", state["messages"][-1].content)

state['messages'].append(HumanMessage(content="How do I get there from London?"))
print("User:\n", state["messages"][-1].content)
state = app.invoke(state)
print("Assistant:\n", state["messages"][-1].content)